# Projeto 1 - Ciência dos Dados

Nome: Arthur Motta Pereira Gomes

Nome: Enzo Martins Barroso Del Manto

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [18]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from math import *

In [19]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\ZEBINHO\Downloads\INSPER\2° Semestre\CDados\P1\Projeto


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [20]:
filename = 'netflix.xlsx'

In [21]:
file = pd.read_excel(filename)
file.head(5)

,Treinamento,Relevância
0,e eu que comecei a assistir anime por causa da...,0
1,#manifest saíra em 20/09 do peacock e do hulu ...,0
2,se eu tivesse netflix eu seria mais feliz pprt,1
3,assinem a netflix para os seus avós e impeçam ...,0
4,eu nunca vou superar toda vez que eu tiver a o...,1


In [5]:
train = pd.read_excel(filename, sheet_name='Treinamento')
train.head(5)

,Treinamento,Relevância
0,e eu que comecei a assistir anime por causa da...,0
1,#manifest saíra em 20/09 do peacock e do hulu ...,0
2,se eu tivesse netflix eu seria mais feliz pprt,1
3,assinem a netflix para os seus avós e impeçam ...,0
4,eu nunca vou superar toda vez que eu tiver a o...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é um serviço virtual por assinatura que disponibiliza filmes e séries para o usuário assistir. Nós consideramos como relevantes tweets que opinam sobre o serviço da Netflix.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
alpha = 0.5
v = 1e4

#CLEANUP - Função de limpeza muito simples que troca alguns sinais básicos por espaços
def cleanup(text):
    punctuation = '[!-.:?;"\n"()'',_%$<>\|/]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

#FREQWORDS - Definir frequência de cada palavra e definir relevância do tweet
def freqwords(tweet):
    
    prob_tweet_r = 1
    prob_tweet_i = 1
    
    tweets = tweet.split(' ')
    
    for word in tweets:
        if word in freq_re_rel:
            prob_tweet_r *= freq_re_rel[word]
        elif word not in freq_re_rel:
            prob_tweet_r *= (alpha/(freq_ab_rel.sum() + (alpha*v)))
            
        if word in freq_re_irr:
            prob_tweet_i *= freq_re_irr[word]
        elif word not in freq_re_irr:
            prob_tweet_i *= (alpha/(freq_ab_irr.sum() + (alpha*v)))
            
    prob_tweet_rel = prob_tweet_r*p_rel
    prob_tweet_irr = prob_tweet_i*p_irr
    
    if max(log(prob_tweet_rel, 10), log(prob_tweet_irr, 10)) == log(prob_tweet_rel, 10): 
        return 'Relevante'
    if max(log(prob_tweet_rel, 10), log(prob_tweet_irr, 10)) == log(prob_tweet_irr, 10):
        return 'Irrelevante'

In [22]:
#Limpando tweets de treinamento
train['Treinamento'] = train['Treinamento'].apply(cleanup)

#Transformando caracteres dos tweets em minúsculos
train['Treinamento'] = train['Treinamento'].str.lower()

#Transformando Relevância em uma variável qualitativa ordinal
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Irrelevante', 'Relevante']

#DataFrames separados por relevância
relevante = train['Relevância'] == 'Relevante'
irrelevante = train['Relevância'] == 'Irrelevante'
df_relevante = train.loc[relevante, :]
df_irrelevante = train.loc[irrelevante, :]

#Separando DataFrames em listas de palavras e em listas de tweets
tweets_relevantes = []
lista_words_rel = []
for tweet in df_relevante['Treinamento']:
    tweets_relevantes.append(tweet.split(' '))
for word in tweets_relevantes:
    lista_words_rel += word
tweets_irrelevantes = []
lista_words_irr = []
for tweet in df_irrelevante['Treinamento']:
    tweets_irrelevantes.append(tweet.split(' '))
for word in tweets_irrelevantes:
    lista_words_irr += word

#Transformando as listas de palavras em pd.Series
serie_relevante = pd.Series(lista_words_rel)
serie_irrelevante = pd.Series(lista_words_irr)

#Probabilidade de ser relevante/irrelevante
p_rel = len(tweets_relevantes) / len(train["Treinamento"])
p_irr = len(tweets_irrelevantes) / len(train["Treinamento"])

#Relevância = 0
freq_ab_irr = serie_irrelevante.value_counts() 
freq_re_irr = (freq_ab_irr + alpha)/(freq_ab_irr.sum() + (alpha*v))

#Relevância = 1
freq_ab_rel = serie_relevante.value_counts() 
freq_re_rel = (freq_ab_rel + alpha)/(freq_ab_rel.sum() + (alpha*v))

#Aplicando função na planilha de treino
train['Classificador'] = train['Treinamento'].apply(freqwords)

#Gerando tabela cruzada da planilha de treino
tabela_train = pd.crosstab(train['Relevância'], train['Classificador'], normalize='all')
tabela_train

Classificador,Irrelevante,Relevante
Relevância,,
Irrelevante,0.570000,0.000000
Relevante,0.036667,0.393333


In [23]:
print('O acerto do classificador na planilha de treinos foi de: {0:.2f}%'.format((tabela_train.iloc[0,0]+tabela_train.iloc[1,1])*100))

O acerto do classificador na planilha de treinos foi de: 96.33%


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
#Lendo planilha Teste
test = pd.read_excel(filename, sheet_name = 'Teste')

#Limpando tweets de teste
test['Teste'] = test['Teste'].apply(cleanup)

#Transformando caracteres dos tweets em minúsculos
test['Teste'] = test['Teste'].str.lower()

#Transformando Relevância em uma variável qualitativa ordinal
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = ['Irrelevante', 'Relevante']

#Aplicando função na planilha de testes
test['Classificador'] = test['Teste'].apply(freqwords)

#Gerando tabela cruzada da planilha de testes
tabela_test = pd.crosstab(test['Relevância'], test['Classificador'], normalize='all')
tabela_test

Classificador,Irrelevante,Relevante
Relevância,,
Irrelevante,0.41,0.07
Relevante,0.32,0.20


In [25]:
print('O acerto do classificador na planilha de testes foi de: {0}%'.format((tabela_test.iloc[0,0]+tabela_test.iloc[1,1])*100))

O acerto do classificador na planilha de testes foi de: 61.0%


## ___
### Concluindo

In [26]:
print(f'Porcentagem de mensagens relevantes que foram classificadas como relevantes: {(tabela_test.iloc[1,1])*192.3:g}%')
print(f'Porcentagem de mensagens relevantes que não foram classificadas como relevantes: {(tabela_test.iloc[1,0])*192.3:g}%')

Porcentagem de mensagens relevantes que foram classificadas como relevantes: 38.46%
Porcentagem de mensagens relevantes que não foram classificadas como relevantes: 61.536%


In [27]:
print(f'Porcentagem de mensagens irrelevantes que foram classificadas como irrelevantes: {(tabela_test.iloc[0,0])*208.3:g}%')
print()
print(f'Porcentagem de mensagens irrelevantes que não foram classificadas como irrelevantes: {(tabela_test.iloc[0,1])*208.3:g}%')

Porcentagem de mensagens irrelevantes que foram classificadas como irrelevantes: 85.403%

Porcentagem de mensagens irrelevantes que não foram classificadas como irrelevantes: 14.581%


CONCLUSÃO:

Como pode ser visto, o programa teve uma taxa razoavelmente alta de acertos para os tweets classificados como irrelevantes, enquanto para os tweets classificados como relevantes nem tanto. Levando-se em consideração a quantidade dos tweets marcados manualmente como relevantes e irrelevantes, a taxa de acerto dos tweets classificados como irrelevantes foi de 85.403%, enquanto a taxa de acerto dos tweets classificados como relevantes foi de apenas 38.46%.

Essa incompatibilidade no classificador manual e de Naive-Bayes, pode ser justificada por diversos fatores. Entre eles, está o fato de que grande parte dos tweets relevantes são respostas à outros tweets, e portanto, aparecem com um link ao lado; esses links também estão muito presentes nos tweets irrelevantes que são maioria na dataframe e isso pode estar prejudicando o funcionamento do classificador. Adicionalmente, a utilização de gírias/slangs e a forma como os tweets são escritos também podem estar afetando o resultado final, visto que existem palavras que possuem sentido ambíguo e podem ser utilizadas positivamente ou negativamente, o classificador, ao vê-las em tanto tweets classificados relevantes e irrelevantes, “se confunde”.

Dessa forma, para se melhorar a acurácia do nosso classificador, pode-se criar uma função que elimina os links de resposta à tweets e analisa o contexto da frase como um todo, assim não fazendo com que links e a utilização de palavras com sentido ambíguo não afetem a classificação final dos tweets.


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**